In [2]:
# Tao csv file gom : ma san pham, name, gia, so luong ban, rating, image 1, image 2, image3

In [100]:
import requests
import json
import pandas as pd
from datetime import datetime

In [101]:
path = r'D:\0. Start of a DEV\test_data_planner.json'
tasks_details = []
buckets_details = []

In [102]:
def read_json_file(path):
    with open(path, 'r', encoding = 'utf-8') as f:        
        return f.read()

In [103]:
def convert_to_datetime(timestamp):
    if not timestamp:
        return None
    timestamp = sample.replace('T', ' ')
    timestamp = sample.replace('Z', '')
    return timestamp

In [104]:
data_file = read_json_file(path)
data_dict = json.loads(data_file)
list(data_dict.keys())

['Green_fields', 'Renesas']

In [105]:
for project_name in data_dict.keys():
    tasks_dict = data_dict[project_name]['tasks']
    buckets_dict = data_dict[project_name]['buckets']
    
    for task in tasks_dict:
        task_id = task['id']
        plan_id = task['planId']
        task_name = task['title']
        user_ids = list(task['assignments'].keys())
        labels = list(task['appliedCategories'].keys())
        start_date = task.get('createdDateTime')
        due_date = task.get('dueDateTime')
        bucket_id = task['bucketId']
        tasks_details.append({'project_name': project_name,
                             'project_id': plan_id,
                             'card_name': task_name,
                             'label': labels,
                             'user_id': user_ids,
                             'created_date': start_date,
                             'due_date': due_date,
                             'bucket_id': bucket_id})
    
    for bucket in buckets_dict:
        bucket_name = bucket['name']
        bucket_id = bucket['id']
        buckets_details.append({'bucket_name': bucket_name,
                               'bucket_id': bucket_id})
        

In [106]:
task_df = pd.DataFrame(tasks_details)
bucket_df = pd.DataFrame(buckets_details)
bucket_df.dtypes

bucket_name    object
bucket_id      object
dtype: object

In [107]:
planner_data = pd.merge(task_df, bucket_df, on = ['bucket_id'])
planner_data.drop(columns = 'bucket_id', axis = 1, inplace = True)

In [119]:
planner_data_ex = planner_data.explode('label')
planner_data_ex = planner_data_ex.explode('user_id')

In [120]:
planner_data_ex['created_date'] = planner_data_ex['created_date'].map(convert_to_datetime)
planner_data_ex['due_date'] = planner_data_ex['due_date'].map(convert_to_datetime)
#planner_data_ex['due_date'] = planner_data_ex.apply(lambda x: convert_to_datetime(x['due_date']), axis = 1)

In [131]:
planner_data_ex['insert_time'] = datetime.now()

In [132]:
planner_data_ex.to_csv('planner_data_ex.csv', index = False, encoding = 'utf-8-sig')